Paquetes necesarios

In [ ]:
import cv2
from ultralytics import YOLO

# Carga del modelo
model = YOLO('yolo11n.pt')  # Contenedores

# Para un vídeo 
filename = "videoplayback.mp4"
results = model.track(source=filename, stream=True)  # Usamos 'stream=True' para obtener los frames uno a uno

# Inicialización de la ventana y reproducción de frames
for result in results:
    frame = result.orig_img  # Obtener el frame original del resultado

    # Dibujar las detecciones en el frame
    if result.boxes is not None:  # Verifica si hay detecciones
        for box in result.boxes:  # Accede a cada caja detectada
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas de la caja
            label = box.cls  
            confidence = box.conf.item()  

            # Texto de la etiqueta 
            text = f"{model.names[int(label)]} {confidence:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Dibujar la caja en verde
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Dibujar el texto

    # Redimensionar el frame y mostrarlo
    frame_resized = cv2.resize(frame, (960, 540))  # Ajustar el tamaño de la ventana
    cv2.imshow("Video Resized", frame_resized)

    # Detener la ejecución si se presiona la tecla Escape
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO

# Definir la ubicación del archivo YAML de datos
data_yaml_path = 'C:/Users/joele/anaconda3/envs/VC_P4/miarchivo.yml'  # Cambia esto a la ruta de tu archivo YAML

# Cargar un modelo YOLOv8 base para entrenar desde cero
model = YOLO('yolov8n.yaml')  # Puedes usar el archivo de configuración de modelo YOLOv8

# Configuración y entrenamiento
model.train(data=data_yaml_path, imgsz=416, batch=4, device='cpu', epochs=40, name="entrenamiento_definitivo")  # Nombre personalizado para el archivo de salida

print("Entrenamiento completo.")


In [ ]:
import cv2
import easyocr
import pandas as pd
from ultralytics import YOLO
from collections import defaultdict
import numpy as np

# Crear un lector de EasyOCR
reader = easyocr.Reader(['es'])

model_general = YOLO('yolo11n.pt')
model_path = "C:/Users/noelp/anaconda3/envs/VC_P4/runs/detect/entrenamiento_definitivo/weights/best.pt"
model_plates = YOLO(model_path)

# Para un vídeo o imagen
filename = "video.mp4"
results_stream = model_general.track(source=filename, stream=True)

# Inicialización de la lista para los resultados
results_list = []
frame_count = 0

# Inicialización de contadores
total_cars = 0
total_people = 0

# Diccionarios para rastrear objetos
tracked_cars = {}
tracked_people = {}

# Función para calcular la distancia entre dos puntos
def calculate_distance(box1, box2):
    center1 = np.array([(box1[0] + box1[2]) / 2, (box1[1] + box1[3]) / 2])
    center2 = np.array([(box2[0] + box2[2]) / 2, (box2[1] + box2[3]) / 2])
    return np.linalg.norm(center1 - center2)

# Inicialización de la ventana y reproducción de frames
for result in results_stream:
    frame = result.orig_img
    frame_count += 1

    # Detección de coches y personas en el frame
    if result.boxes is not None:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = model_general.names[int(box.cls)]
            confidence = box.conf.item()

            # Mostrar solo coches y personas
            if label == 'car':
                # Comprobar si ya existe el coche en el seguimiento
                identifier = None
                for id, (prev_box, _) in tracked_cars.items():
                    if calculate_distance((x1, y1, x2, y2), prev_box) < 50:  # Umbral de distancia
                        identifier = id
                        break

                if identifier is None:
                    identifier = f"car_{total_cars + 1}"
                    total_cars += 1
                tracked_cars[identifier] = ((x1, y1, x2, y2), confidence)

                text = f"{label} {confidence:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Guardar resultados en la lista
                results_list.append({
                    'frame': frame_count,
                    'tipo_objeto': label,
                    'confianza': confidence,
                    'identificador_tracking': identifier,
                    'x1': x1,
                    'y1': y1,
                    'x2': x2,
                    'y2': y2,
                    'matricula_en_su_caso': None,
                    'confianza_matricula': None,
                    'mx1': None,
                    'my1': None,
                    'mx2': None,
                    'my2': None,
                    'texto_matricula': None
                })

            elif label == 'person':
                # Comprobar si ya existe la persona en el seguimiento
                identifier = None
                for id, (prev_box, _) in tracked_people.items():
                    if calculate_distance((x1, y1, x2, y2), prev_box) < 50:  # Umbral de distancia
                        identifier = id
                        break

                if identifier is None:
                    identifier = f"person_{total_people + 1}"
                    total_people += 1
                tracked_people[identifier] = ((x1, y1, x2, y2), confidence)

                text = f"{label} {confidence:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                # Guardar resultados en la lista
                results_list.append({
                    'frame': frame_count,
                    'tipo_objeto': label,
                    'confianza': confidence,
                    'identificador_tracking': identifier,
                    'x1': x1,
                    'y1': y1,
                    'x2': x2,
                    'y2': y2,
                    'matricula_en_su_caso': None,
                    'confianza_matricula': None,
                    'mx1': None,
                    'my1': None,
                    'mx2': None,
                    'my2': None,
                    'texto_matricula': None
                })

    # Detección de matrículas
    results_plates = model_plates(frame)
    for result_plates in results_plates:
        if result_plates.boxes is not None:
            for box in result_plates.boxes:
                mx1, my1, mx2, my2 = map(int, box.xyxy[0])
                plate_region = frame[my1:my2, mx1:mx2]

                # Usar EasyOCR para leer el texto de la matrícula
                results = reader.readtext(plate_region)
                plate_text = " ".join([text for (_, text, _) in results])

                # Mostrar la matrícula
                text_plate = f"Matricula: {plate_text.strip()}"
                cv2.rectangle(frame, (mx1, my1), (mx2, my2), (255, 0, 0), 2)
                cv2.putText(frame, text_plate, (mx1, my1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                # Actualizar la lista con la información de la matrícula
                for res in results_list:
                    if res['frame'] == frame_count and res['tipo_objeto'] == 'car':
                        res.update({
                            'matricula_en_su_caso': plate_text.strip(),
                            'confianza_matricula': None,
                            'mx1': mx1,
                            'my1': my1,
                            'mx2': mx2,
                            'my2': my2,
                            'texto_matricula': text_plate.strip()
                        })
                        break

    # Redimensionar el frame y mostrarlo
    frame_resized = cv2.resize(frame, (960, 540))
    cv2.imshow("Video Resized", frame_resized)

    # Detener la ejecución si se presiona la tecla Escape
    if cv2.waitKey(20) & 0xFF == 27:
        break

# Guardar los resultados en un archivo CSV
df = pd.DataFrame(results_list)
df.to_csv('resultados_deteccion.csv', index=False)

# Mostrar el total de coches y personas detectados
print(f'Total de coches detectados: {total_cars}')
print(f'Total de personas detectadas: {total_people}')

# Finalizar la ventana de visualización
cv2.destroyAllWindows()
